In [15]:
import sagemaker
import boto3
import os

from sagemaker.predictor import Predictor
from sagemaker.pytorch import PyTorchModel

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

class ImagePredictor(Predictor):
  def __init__(self, endpoint_name, sagemaker_session):
    super(ImagePredictor, self).__init__(
      endpoint_name,
      sagemaker_session=sagemaker_session,
      serializer=sagemaker.serializers.IdentitySerializer("image/jpeg"),
      deserializer=sagemaker.deserializers.JSONDeserializer())
    
pytorch_model = PyTorchModel(model_data="s3://amazonbin/deploy/model.tar.gz",
                             image_uri="763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-inference:1.11.0-cpu-py38-ubuntu20.04-sagemaker",
                             role=role,
                             entry_point='model/code/inference.py',
                             py_version='py3',
                             framework_version='1.8.0',
                             predictor_cls=ImagePredictor)

predictor = pytorch_model.deploy(initial_instance_count=1, instance_type='ml.m5.large')

------!

In [ ]:
from PIL import Image
import io
buf = io.BytesIO()
Image.open("00024.jpg").save(buf, format="JPEG")

response = predictor.predict(buf.getvalue())

In [12]:
predictor.delete_endpoint()